## User

### concat

1. 단순 concat

In [1]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

df = pd.read_csv("user_patent_info.csv")

row = df.iloc[0] 
ipc = row['ipc_code'] if isinstance(row['ipc_code'], str) else ''

text = (
    f"Country: {row.get('country', '')}\n"
    f"Date: {row.get('date', '')}\n"
    f"Title: {row.get('title', '')}\n"
    f"Applicant: {row.get('applicant', '')}\n"
    f"IPC: {ipc}\n"
    f"Abstract: {row.get('abstract', '')}"
)

In [20]:
text

'Country: US\nDate: 2024-05-02\nTitle: PROPIONIBACTERIUM ACNES PROPHYLACTIC AND THERAPEUTIC IMMUNE TREATMENT\nApplicant: Origimm Biotechnology GmbH, Wien\nIPC: A61K 39/05, AGIP 17/10, AGIP 31/04, CO7K 14/195\nAbstract: The present invention discloses a vaccine comprising one or more of Dermatan sulfate-binding adhesin 1 of P. acnes (DsA1 polypeptide), Dermatan sulfate-binding adhesin 2 of P. acnes (DsA2 polypeptide), and putative iron-transport protein (PITP) polypeptide of P. acnes, and/or a fragment and/or derivative of DsAl and/or DsA2 and/or PITP, wherein the DsA1 polypeptide and the DsA2 polypeptide comprise from N- to C-terminus an N-terminal swapping region (“NSR”), a first conserved sub-domain (“CSD1”), a first swapping region (“SR1”), a second conserved sub-domain (“CSD2”), a second swapping region (“SR2”), a third conserved sub-domain (“CSD3”), a Pro-Thr repeat containing region (“PT repeat region”), and a C-terminal region (“CTR”), and wherein the PITP polypeptide comprises 

### user patent -> FAISS vector DB

In [2]:
response = client.embeddings.create(
    model="text-embedding-3-large",  # or text-embedding-3-small (더 저렴)
    input=text
)

embedding_vector = response.data[0].embedding

In [3]:
import faiss
import numpy as np

dimension = len(embedding_vector)
index = faiss.IndexFlatL2(dimension)  # L2 거리 기반
index.add(np.array([embedding_vector]))  # 벡터 1개 추가

print(f"✅ Index에 벡터 추가 완료. 총 벡터 수: {index.ntotal}")

faiss.write_index(index, "user_patent_index.faiss")
print("✅ FAISS 인덱스 저장 완료: user_patent_index.faiss")

✅ Index에 벡터 추가 완료. 총 벡터 수: 1
✅ FAISS 인덱스 저장 완료: patent_index.faiss


## NAICS

### NAICS parse

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import faiss
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

df_naics = pd.read_excel("data/NAICS_description.xlsx")

def extract_activity(desc):
    if pd.isna(desc):
        return None
    match = re.search(r"engaged in (.*)", desc, re.IGNORECASE)
    return match.group(1).strip() if match else desc.strip()

df_naics["activity"] = df_naics["Description"].apply(extract_activity)
df_naics = df_naics.dropna(subset=["activity"])

df_naics["combined_text"] = (
    df_naics["2022 NAICS US Title"].fillna('') + " — " + df_naics["activity"].fillna('')
)

embeddings = []
batch_size = 50  

for i in range(0, len(df_naics), batch_size):
    batch = df_naics["combined_text"].iloc[i:i+batch_size].tolist()
    response = client.embeddings.create(
        model="text-embedding-3-large",
        input=batch
    )
    batch_embeddings = [np.array(e.embedding, dtype="float32") for e in response.data]
    embeddings.extend(batch_embeddings)

# numpy 배열로 변환
embeddings = np.vstack(embeddings)

print(f"✅ NAICS 임베딩 완료: shape={embeddings.shape}")

In [16]:
# --- 5️⃣ 코사인 유사도용 FAISS 인덱스 생성 ---
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner Product = Cosine similarity
faiss.normalize_L2(embeddings)
index.add(embeddings)

# --- 6️⃣ 메타데이터 함께 저장 (코드, 제목, 설명 등) ---
meta_df = df_naics[["naics_code", "naics_title", "description"]].copy()
meta_df.to_csv("naics_metadata.csv", index=False, encoding="utf-8-sig")

# --- 7️⃣ 인덱스 저장 ---
faiss.write_index(index, "naics_index.faiss")

print("✅ NAICS 인덱스 저장 완료: naics_index.faiss")
print("✅ 메타데이터 저장 완료: naics_metadata.csv")

✅ NAICS 인덱스 저장 완료: naics_index.faiss
✅ 메타데이터 저장 완료: naics_metadata.csv


In [18]:
import faiss
import numpy as np
import pandas as pd

# --- 1️⃣ 인덱스 불러오기 ---
user_index = faiss.read_index("user_patent_index.faiss")
naics_index = faiss.read_index("naics_index.faiss")

# --- 2️⃣ 유저 임베딩 추출 ---
user_emb = np.zeros((1, user_index.d), dtype="float32")
user_emb[0] = user_index.reconstruct(0)   # 첫 번째 유저 특허 임베딩 벡터 추출
faiss.normalize_L2(user_emb)  # 코사인 유사도용 정규화

# --- 3️⃣ NAICS 메타데이터 로드 ---
df_naics = pd.read_csv("naics_metadata.csv")

# --- 4️⃣ 유사도 검색 (Top 5) ---
k = 5
D, I = naics_index.search(user_emb, k)
top5 = df_naics.iloc[I[0]]

# --- 5️⃣ 결과 출력 ---
print("\n🔎 유사한 NAICS TOP 5")
for rank, (code, title, desc, score) in enumerate(zip(
    top5["naics_code"],
    top5["naics_title"],
    top5["description"],
    D[0]
)):
    print(f"{rank+1}. [{code}] {title} — Score: {score:.3f}")
    print(f"   ↳ {desc}\n")



🔎 유사한 NAICS TOP 5
1. [325414] Biological Product (except Diagnostic) Manufacturing — Score: 0.319
   ↳ This U.S. industry comprises establishments primarily engaged in manufacturing vaccines, toxoids, blood fractions, and culture media of plant or animal origin (except diagnostic).

Cross-References. Establishments primarily engaged in--

2. [325412] Pharmaceutical Preparation Manufacturing — Score: 0.257
   ↳ This U.S. industry comprises establishments primarily engaged in manufacturing in-vivo diagnostic substances and pharmaceutical preparations (except biological) intended for internal and external consumption in dose forms, such as ampoules, tablets, capsules, vials, ointments, powders, solutions, and suspensions.

Cross-References. Establishments primarily engaged in--

3. [325620] Toilet Preparation Manufacturing — Score: 0.224
   ↳ This industry comprises establishments primarily engaged in preparing, blending, compounding, and packaging toilet preparations, such as perfumes, 